In [5]:
import pandas as pd
import os

# Folder path containing CSVs
data_folder = r"E:\PKNU_Data_2025\Aanlysis\Anlysis_14nos\data\Re_annotated_data"

# List CSV files
csv_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]

# Helper to remove outliers using IQR
def remove_outliers(data):
    series = pd.Series(data).dropna()
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    return series[(series >= (Q1 - 1.5 * IQR)) & (series <= (Q3 + 1.5 * IQR))]

# Storage for metrics
all_step_times, all_stride_L, all_stride_R = [], [], []
all_stance_L, all_stance_R = [], []

for file in csv_files:
    try:
        df = pd.read_csv(os.path.join(data_folder, file))
        df['Timestamp'] = pd.to_datetime(df['Timestamp'].astype(str).str.strip("'"), errors='coerce')
        df = df.dropna(subset=["Timestamp"])

        # Step Time
        heel_strikes = df[(df["L_GAIT"] == "HES") | (df["R_GAIT"] == "HES")].copy()
        heel_strikes["Foot"] = heel_strikes.apply(lambda row: "L" if row["L_GAIT"] == "HES" else "R", axis=1)
        heel_strikes["Step_Time"] = heel_strikes["Timestamp"].diff()
        heel_strikes["Prev_Foot"] = heel_strikes["Foot"].shift(1)
        valid_steps = heel_strikes[heel_strikes["Foot"] != heel_strikes["Prev_Foot"]]
        all_step_times.extend(valid_steps["Step_Time"].dt.total_seconds().dropna().tolist())

        # Stride Time
        L_HES = df[df["L_GAIT"] == "HES"]["Timestamp"]
        R_HES = df[df["R_GAIT"] == "HES"]["Timestamp"]
        all_stride_L.extend(L_HES.diff(2).dt.total_seconds().dropna().tolist())
        all_stride_R.extend(R_HES.diff(2).dt.total_seconds().dropna().tolist())

        # Stance Time
        L_HES_df = df[df["L_GAIT"] == "HES"][["Timestamp"]].rename(columns={"Timestamp": "Heel_Strike_L"}).sort_values("Heel_Strike_L")
        L_TOF_df = df[df["L_GAIT"] == "TOF"][["Timestamp"]].rename(columns={"Timestamp": "Toe_Off_L"}).sort_values("Toe_Off_L")
        if not L_HES_df.empty and not L_TOF_df.empty:
            L_merge = pd.merge_asof(L_HES_df, L_TOF_df, left_on="Heel_Strike_L", right_on="Toe_Off_L", direction="forward")
            L_merge["Stance_Time_L"] = (L_merge["Toe_Off_L"] - L_merge["Heel_Strike_L"]).dt.total_seconds()
            all_stance_L.extend(L_merge["Stance_Time_L"].dropna()[L_merge["Stance_Time_L"] > 0])

        R_HES_df = df[df["R_GAIT"] == "HES"][["Timestamp"]].rename(columns={"Timestamp": "Heel_Strike_R"}).sort_values("Heel_Strike_R")
        R_TOF_df = df[df["R_GAIT"] == "TOF"][["Timestamp"]].rename(columns={"Timestamp": "Toe_Off_R"}).sort_values("Toe_Off_R")
        if not R_HES_df.empty and not R_TOF_df.empty:
            R_merge = pd.merge_asof(R_HES_df, R_TOF_df, left_on="Heel_Strike_R", right_on="Toe_Off_R", direction="forward")
            R_merge["Stance_Time_R"] = (R_merge["Toe_Off_R"] - R_merge["Heel_Strike_R"]).dt.total_seconds()
            all_stance_R.extend(R_merge["Stance_Time_R"].dropna()[R_merge["Stance_Time_R"] > 0])

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Remove outliers
filtered_step = remove_outliers(all_step_times)
filtered_stride_L = remove_outliers(all_stride_L)
filtered_stride_R = remove_outliers(all_stride_R)
filtered_stance_L = remove_outliers(all_stance_L)
filtered_stance_R = remove_outliers(all_stance_R)

# Create summary
def format_mean_std(series):
    return f"{series.mean():.3f} ± {series.std():.3f}"

results = {
    "Step Time (s)": format_mean_std(filtered_step),
    "Stride Time Left (s)": format_mean_std(filtered_stride_L),
    "Stride Time Right (s)": format_mean_std(filtered_stride_R),
    "Stance Time Left (s)": format_mean_std(filtered_stance_L),
    "Stance Time Right (s)": format_mean_std(filtered_stance_R),
}

# Print results
for k, v in results.items():
    print(f"{k}: {v}")


Step Time (s): 0.428 ± 0.246
Stride Time Left (s): 0.957 ± 0.618
Stride Time Right (s): 1.008 ± 0.724
Stance Time Left (s): 0.681 ± 0.263
Stance Time Right (s): 0.734 ± 0.194


In [1]:
import pandas as pd
import os

# Folder path containing CSVs
data_folder = r"E:\PKNU_Data_2025\Aanlysis\Anlysis_10nos\Data\Re_annotated_data_1"

# List CSV files
csv_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]

# Helper to remove outliers using IQR
def remove_outliers(data):
    series = pd.Series(data).dropna()
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    return series[(series >= (Q1 - 1.5 * IQR)) & (series <= (Q3 + 1.5 * IQR))]

# Storage for metrics
all_step_times, all_stride_L, all_stride_R = [], [], []
all_stance_L, all_stance_R = [], []

for file in csv_files:
    try:
        df = pd.read_csv(os.path.join(data_folder, file))
        df['Timestamp'] = pd.to_datetime(df['Timestamp'].astype(str).str.strip("'"), errors='coerce')
        df = df.dropna(subset=["Timestamp"])

        # Step Time
        heel_strikes = df[(df["L_GAIT"] == "HES") | (df["R_GAIT"] == "HES")].copy()
        heel_strikes["Foot"] = heel_strikes.apply(lambda row: "L" if row["L_GAIT"] == "HES" else "R", axis=1)
        heel_strikes["Step_Time"] = heel_strikes["Timestamp"].diff()
        heel_strikes["Prev_Foot"] = heel_strikes["Foot"].shift(1)
        valid_steps = heel_strikes[heel_strikes["Foot"] != heel_strikes["Prev_Foot"]]
        all_step_times.extend(valid_steps["Step_Time"].dt.total_seconds().dropna().tolist())

        # Stride Time
        L_HES = df[df["L_GAIT"] == "HES"]["Timestamp"]
        R_HES = df[df["R_GAIT"] == "HES"]["Timestamp"]
        all_stride_L.extend(L_HES.diff(2).dt.total_seconds().dropna().tolist())
        all_stride_R.extend(R_HES.diff(2).dt.total_seconds().dropna().tolist())

        # Stance Time
        L_HES_df = df[df["L_GAIT"] == "HES"][["Timestamp"]].rename(columns={"Timestamp": "Heel_Strike_L"}).sort_values("Heel_Strike_L")
        L_TOF_df = df[df["L_GAIT"] == "TOF"][["Timestamp"]].rename(columns={"Timestamp": "Toe_Off_L"}).sort_values("Toe_Off_L")
        if not L_HES_df.empty and not L_TOF_df.empty:
            L_merge = pd.merge_asof(L_HES_df, L_TOF_df, left_on="Heel_Strike_L", right_on="Toe_Off_L", direction="forward")
            L_merge["Stance_Time_L"] = (L_merge["Toe_Off_L"] - L_merge["Heel_Strike_L"]).dt.total_seconds()
            all_stance_L.extend(L_merge["Stance_Time_L"].dropna()[L_merge["Stance_Time_L"] > 0])

        R_HES_df = df[df["R_GAIT"] == "HES"][["Timestamp"]].rename(columns={"Timestamp": "Heel_Strike_R"}).sort_values("Heel_Strike_R")
        R_TOF_df = df[df["R_GAIT"] == "TOF"][["Timestamp"]].rename(columns={"Timestamp": "Toe_Off_R"}).sort_values("Toe_Off_R")
        if not R_HES_df.empty and not R_TOF_df.empty:
            R_merge = pd.merge_asof(R_HES_df, R_TOF_df, left_on="Heel_Strike_R", right_on="Toe_Off_R", direction="forward")
            R_merge["Stance_Time_R"] = (R_merge["Toe_Off_R"] - R_merge["Heel_Strike_R"]).dt.total_seconds()
            all_stance_R.extend(R_merge["Stance_Time_R"].dropna()[R_merge["Stance_Time_R"] > 0])

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Remove outliers
filtered_step = remove_outliers(all_step_times)
filtered_stride_L = remove_outliers(all_stride_L)
filtered_stride_R = remove_outliers(all_stride_R)
filtered_stance_L = remove_outliers(all_stance_L)
filtered_stance_R = remove_outliers(all_stance_R)

# Create summary
def format_mean_std(series):
    return f"{series.mean():.3f} ± {series.std():.3f}"

results = {
    "Step Time (s)": format_mean_std(filtered_step),
    "Stride Time Left (s)": format_mean_std(filtered_stride_L),
    "Stride Time Right (s)": format_mean_std(filtered_stride_R),
    "Stance Time Left (s)": format_mean_std(filtered_stance_L),
    "Stance Time Right (s)": format_mean_std(filtered_stance_R),
}

# Print results
for k, v in results.items():
    print(f"{k}: {v}")


Step Time (s): 0.431 ± 0.237
Stride Time Left (s): 1.002 ± 0.634
Stride Time Right (s): 1.015 ± 0.709
Stance Time Left (s): 0.708 ± 0.268
Stance Time Right (s): 0.723 ± 0.185
